In [12]:
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os

In [19]:
participant_data_path = './EEG/number'
processed_data_path = './EEG/processed_data'
split_data_path = './EEG/split_data'

In [27]:
ps = os.listdir(participant_data_path)
participants=[]
for p in ps:
    p= p[:3]
    if p not in participants:
        participants.append(p)
        
# participants, len(participants)

In [21]:
options={0:"number",1:"graph"}; option=0
name= f"data_scaled_p_dictionary-{options[option]}"
scaler= "standard_scaled"
dictionary= pickle.load(open(os.path.join(processed_data_path, scaler, f"{name}.pkl"), "rb"))
# os.path.join(processed_data_path, "minmax_scaled", f"data_without_artefact_binary_dictionary.pkl")

In [25]:
# # Sanity check
# for participant in participants:
#     print(np.unique(dictionary[participant][1],return_counts=True))

In [23]:
# Class balancing
class_ratio={0:1.,1:1.}
    
def class_balancing(class_ratio:dict, dictionary:dict,random_seed=99):
    np.random.seed(random_seed)
    
    indices_dictionary={}
    for participant in participants:
        indices_dictionary[participant]=[]
        for label in range(len(class_ratio)):
            indices= np.nonzero(dictionary[participant][1]==label)[0]
            indices_len= int(round(len(indices)*class_ratio[label]))

            np.random.shuffle(indices)

            indices_dictionary[participant].append(indices[:indices_len])
        
    return indices_dictionary
indices_dictionary= class_balancing(class_ratio, dictionary)

## Userfold

In [24]:
if not os.path.exists(split_data_path):
    os.makedirs(split_data_path)
if not os.path.exists(os.path.join(split_data_path, scaler)):
    os.makedirs(os.path.join(split_data_path, scaler))

userfold_dict={}
for participant in participants:

    x, y= dictionary[participant]
    
    X=[]
    Y=[]
    for indices in indices_dictionary[participant]:
        X.append(x[indices])
        Y.append(y[indices])
    
    userfold_dict[participant]= (np.concatenate(X),np.concatenate(Y))
    
pickle.dump(userfold_dict,open(os.path.join(split_data_path, scaler,f"userfold_{name}.pkl"),"wb"))    

In [26]:
# # sanity check
# for participant in participants:
#     print(userfold_dict[participant][0].shape)
#     print(np.unique(userfold_dict[participant][1]))